# Cleaning data to sum up weekly store's sales rather than departments'


In [1]:
import pandas as pd


In [2]:
ls *.csv

features.csv  sampleSubmission.csv  stores.csv  test.csv  train.csv


In [3]:
# Getting a list of stores
stores = pd.read_csv('stores.csv')
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [4]:
store_id_list = list(stores['Store'])
# print(store_id_list)

In [5]:
# Getting the sales data
train_csv = pd.read_csv('train.csv')
train_csv.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [6]:
# Remove isHoliday column since it does exist in the features csv file
train_csv = train_csv.loc[:, ['Store', 'Dept', 'Date', 'Weekly_Sales']]
train_csv.head()

,Store,Dept,Date,Weekly_Sales
0,1,1,2010-02-05,24924.50
1,1,1,2010-02-12,46039.49
2,1,1,2010-02-19,41595.55
3,1,1,2010-02-26,19403.54
4,1,1,2010-03-05,21827.90


In [7]:
# Making a list of data frames that represent each store with its departments weekly sales
store_df_list = []
for store_id in store_id_list:
    store_list = train_csv[train_csv.Store == store_id]
    store_df_list.append(store_list)
    
store_df_list[43].head()

,Store,Dept,Date,Weekly_Sales
404764,44,1,2010-02-05,6871.20
404765,44,1,2010-02-12,12315.65
404766,44,1,2010-02-19,7751.11
404767,44,1,2010-02-26,6014.71
404768,44,1,2010-03-05,6120.60


In [8]:
# Getting unique dates for the sales data
store_list = store_df_list[0]
dates_list = store_list['Date']
print(len(dates_list))
unique_dates_list = dates_list.unique()
print(len(unique_dates_list))


10244
143


Generate a list of weekly sales for each 
by iterating over dates to get a list of department's sales, summing it up and
then assign the sum to the specific date and store number

In [9]:
# Making a dataframe to add data to
columns = ['Store', 'Date', 'Weekly_Sales']
generated_data_row_list = []
# Iterating over list of unique store dataframes
for store_df in store_df_list:
    # Iterate over list of dates and then get rows of departments
    store_id = store_df.iloc[0].Store
    for week_date in unique_dates_list:
        departments_weekly_sales_series = store_df[store_df.Date == week_date].loc[:,'Weekly_Sales']
        weekly_sales_sum = departments_weekly_sales_series.sum()
        data_row = [store_id, week_date, weekly_sales_sum]
        generated_data_row_list.append(data_row)

generated_train_df = pd.DataFrame(generated_data_row_list, columns = columns)

In [14]:
generated_train_df.head()

,Store,Date,Weekly_Sales
0,1,2010-02-05,1643690.90
1,1,2010-02-12,1641957.44
2,1,2010-02-19,1611968.17
3,1,2010-02-26,1409727.59
4,1,2010-03-05,1554806.68


In [15]:
generated_train_df.describe()

,Store,Weekly_Sales
count,6435.000000,6.435000e+03
mean,23.000000,1.046965e+06
std,12.988182,5.643666e+05
min,1.000000,2.099863e+05
25%,12.000000,5.533501e+05
50%,23.000000,9.607460e+05
75%,34.000000,1.420159e+06
max,45.000000,3.818686e+06


In [16]:
generated_train_df.to_csv('clean_data/weekly_store_sales.csv', index=False)